In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../data/total_reviews.csv')

In [7]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [9]:
df['asin'].unique()

array(['B08267BBJT', 'B08268F6XN', 'B08267X3LH', 'B079JFK22D',
       'B07QW531W2', 'B07TJTQDYG', 'B085HB8QVX', 'B07R2KK5P7',
       'B07Y475MD3', 'B07F27PK2M', 'B085FZFVQV', 'B072R2SWXX',
       'B0828KRQZ3', 'B0821ZNWKW', 'B07QV15B3W', 'B07MWYYDTM',
       'B07RSJMS76', 'B07D1LMMDD', 'B0855TM65T', 'B07NTXYFBV',
       'B086L3Q8YX', 'B085FT4YR1', 'B085XT3GTW', 'B07P9XZPYG',
       'B0824WB5ST', 'B07HQ7QV7W', 'B083ZMYF55', 'B07FQD7PZ5',
       'B07TWDR7VJ', 'B085DV7VZK', 'B086GTFGPP', 'B07Q6PZ2F4',
       'B082W54KQK'], dtype=object)